# WordPiece Tokenizer

Pada kode ini, diberikan implementasi dan contoh penggunaan algoritma WordPiece (sumber: [HuggingFace](https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt))

[Petunjuk instalasi package](https://huggingface.co/docs/transformers/installation)

In [209]:
# Import semua packages yang diperlukan.
from transformers import AutoTokenizer
from collections import defaultdict

In [210]:
# Instantiate pre-tokenizer untuk membagi dokumen menjadi individual words
# berdasarkan keberadaan spasi
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [211]:
# Membaca corpus
with open('idwiki_index.txt') as raw_text:
    corpus = raw_text.readlines()

In [212]:
# Menghitung frekuensi tiap kata pada corpus
word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, _ in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

In [213]:
# Melihat karakter apa saja yang ada pada corpus
alphabet = []
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()

print(alphabet)

['##0', '##4', '##5', '##7', '##9', '##A', '##B', '##C', '##H', '##I', '##M', '##N', '##O', '##R', '##T', '##V', '##W', '##X', '##Y', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##j', '##k', '##l', '##m', '##n', '##o', '##p', '##q', '##r', '##s', '##t', '##u', '##v', '##w', '##x', '##y', '##z', '##á', '##ã', '##ë', '##í', '##ñ', '##ö', '##ú', '##ā', '##ē', '##ł', '##ō', '##а', '##ṭ', "'", '(', ')', ',', '-', '.', '/', '1', '2', '7', ':', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [214]:
# Mendefinisikan vocabulary yang akan digunakan untuk tokenisasi, termasuk
# special token (di luar alphabet pada corpus)
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()

In [215]:
# Membagi tiap kata menjadi individual character dengan adanya special prefix
# untuk karakter yang bukan awal kata.
splits = {
    word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freqs.keys()
}
print(splits)

{'Asam': ['A', '##s', '##a', '##m'], 'deoksiribonukleat': ['d', '##e', '##o', '##k', '##s', '##i', '##r', '##i', '##b', '##o', '##n', '##u', '##k', '##l', '##e', '##a', '##t'], 'Deoksiribosanukleat': ['D', '##e', '##o', '##k', '##s', '##i', '##r', '##i', '##b', '##o', '##s', '##a', '##n', '##u', '##k', '##l', '##e', '##a', '##t'], 'Anwar': ['A', '##n', '##w', '##a', '##r'], 'Sadat': ['S', '##a', '##d', '##a', '##t'], 'Azhar': ['A', '##z', '##h', '##a', '##r'], 'Mansor': ['M', '##a', '##n', '##s', '##o', '##r'], 'Arkeologi': ['A', '##r', '##k', '##e', '##o', '##l', '##o', '##g', '##i'], 'Antropologi': ['A', '##n', '##t', '##r', '##o', '##p', '##o', '##l', '##o', '##g', '##i'], 'Analisis': ['A', '##n', '##a', '##l', '##i', '##s', '##i', '##s'], 'leksikal': ['l', '##e', '##k', '##s', '##i', '##k', '##a', '##l'], 'Bahasa': ['B', '##a', '##h', '##a', '##s', '##a'], 'Indonesia': ['I', '##n', '##d', '##o', '##n', '##e', '##s', '##i', '##a'], 'Biologi': ['B', '##i', '##o', '##l', '##o', '##g',

In [216]:
# Mendefinisikan score untuk menentukan pair mana yang akan di-merge. Hal ini agak
# berbeda dengan BPE yang melakukan merge pada pair yang frekuensinya terbanyak.
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

In [217]:
# Membuat fungsi untuk melakukan merge (dilakukan pada splits)
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [218]:
# Iterasi untuk training, seperti BPE, silakan ubah-ubah bagian vocab_size untuk
# melihat perbedaan hasil tokenisasinya
vocab_size = 100  # Perbesar/perkecil dan lihat bedanya pada hasil tokenisasi
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = "", None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (
        best_pair[0] + best_pair[1][2:]
        if best_pair[1].startswith("##")
        else best_pair[0] + best_pair[1]
    )
    vocab.append(new_token)

In [219]:
# Define fungsi untuk tokenize sample
def encode_word(word):
    tokens = []
    while len(word) > 0:
        i = len(word)
        while i > 0 and word[:i] not in vocab:
            i -= 1
        if i == 0:
            return ["[UNK]"]
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f"##{word}"
    return tokens

def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    encoded_words = [encode_word(word) for word in pre_tokenized_text]
    return sum(encoded_words, [])

In [220]:
# Contoh tokenisasi dengan tokenizer yang sudah dilatih
tokenize('Azhar Mansor')

['A', '##z', '##h', '##a', '##r', 'M', '##a', '##n', '##s', '##o', '##r']